In [11]:
#####
import os
import sys
from datetime import datetime as dt
from datetime import timedelta
sys.path.append('/media/sf__Metr-analyzer/metr_django/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'metr.settings'
import django
django.setup()
from yametr.models import *
write_date=dt(2018,3,1) #первый день нужного месяца
actual_date_id = Pubdate.objects.get(pub_date=write_date).id
fresh_date_id = actual_date_id
print(dt.now())
#####


#https://habrahabr.ru/post/250975/
#https://pythonspot.com/selenium-phantomjs/
#http://forworktests.blogspot.ru/2013/03/selenium-web-driver.html#SeleniumWebDriver-%D0%A1%D0%B2%D0%BE%D0%B9%D1%81%D1%82%D0%B2%D0%BEtext%C2%A0%C2%A0-%D0%B2%D0%BE%D0%B7%D0%B2%D1%80%D0%B0%D1%89%D0%B0%D0%B5%D1%82%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%D1%8D%D0%BB%D0%B5%D0%BC%D0%B5%D0%BD%D1%82%D0%B0web-%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
from python_anticaptcha import AnticaptchaClient, ImageToTextTask
#pip install python-anticaptcha
import requests
from urllib.parse import unquote

user1,pass1="all.kott","all.kott13"
browser=webdriver.Firefox()
# driver = webdriver.Firefox() в driver = webdriver.PhantomJS()
#Остальную часть кода не нужно менять, и браузер не откроется. Для целей отладки используйте 
#driver.save_screenshot('screen.png')

#url = "https://wordstat.yandex.ru/#!/?regions=11316&words=%D1%81%D0%BA%D0%B0%D0%BD%D0%B4%D0%B8%D0%BD%D0%B0%D0%B2%D1%81%D0%BA%D0%B8%D0%B9%20%D1%85%D0%BE%D0%B4%D1%8C%D0%B1%D0%B0"
url="https://wordstat.yandex.ru/#!/history?regions=65"
browser.get(url)
time.sleep(1)



def authorization(user1,pass1):
    try:
        username = browser.find_element_by_id('b-domik_popup-username')
        username.send_keys(user1)
        password = browser.find_element_by_id('b-domik_popup-password')
        password.send_keys(pass1)
        button=browser.find_element_by_class_name('b-domik__button')
        button.find_element_by_class_name('b-form-button__input').click()
    except NoSuchElementException:
        a=0
        #print("no need authorization")

        
        
def getCaptchaLink():
    capt=[]
    while len(capt)<4:
        captcha = browser.find_element_by_class_name("b-popupa__image-td").find_element_by_class_name("b-popupa__image")
        #captcha = browser.find_element_by_xpath(".//img[starts-with(@src, '//ext.captcha.yandex.net')]/@src")
        capt=captcha.get_attribute("outerHTML").split('"')
        time.sleep(1)
        #print(captcha.get_attribute("outerHTML"))
    
    link='https:'+capt[3]
    return link


def downloadCaptcha(url):
    response = requests.get(url)
    if response.status_code == 200:
        with open("/home/alex/Загрузки/captcha.gif", 'wb') as f:
            f.write(response.content)
            time.sleep(1)

def getCaptchaWords():
    apiKeyAntiCaptcha = '445a1a6d2ba30a534c49b43ef090fa35'
    captcha_fp = open('/home/alex/Загрузки/captcha.gif', 'rb')
    client = AnticaptchaClient(apiKeyAntiCaptcha)
    task = ImageToTextTask(captcha_fp)
    job = client.createTask(task)
    job.join()
    return job.get_captcha_text()

def enterCaptcha(captchaWord):
    password = browser.find_element_by_class_name('b-page__captcha-input-td')
    for x in captchaWord:
        password.send_keys(x)
        time.sleep(0.1)
    browser.find_element_by_class_name('b-popupa__form').find_element_by_class_name('b-form-button__content').click()    
    print(captchaWord)
    count()

def GetStat(tr):    
    vl=''
    td=tr.find_elements_by_tag_name("td")
    date=td[0].get_attribute("innerHTML")[:10]
    #b=td[0].get_attribute("innerHTML")
    znach=''
    vals=td[2].find_elements_by_class_name("b-history__number-part")
    for val in vals:
        #print(val)
        vl=vl+val.get_attribute("innerHTML")
    #проверим дату актуализации pub_date_id . заменим ее в случае необходимости
    return date,vl

#глбальный подсчет капчи
class FuncCreator:
    def __init__(self, start_state):
        self.state = start_state

    def __call__(self):
        #print(self.state)
        self.state += 1

count=FuncCreator(0)
countKeys=FuncCreator(0)

def isWordstatStatActual(myKey,pub_date_id):
    try:
        zapr = Zapros.objects.get(zapros_name=myKey)
        zapr.wordstatstat_set.get(pub_date_id=actual_date_id,from_history=1)
        #WordstatStat.objects.get(Zapros__zapros_name=myKey, ws_date=write_date)
        return True
    except WordstatStat.DoesNotExist:
        return False

    
def getInput():
    curl=browser.current_url
    try:
        input_answer=curl[curl.find('words=',)+6:]
        input_answer=unquote(input_answer)
    except:
        input_answer=''
    return input_answer

def checkInput(myKey):
    input_answer=getInput()
    searchForm = browser.find_element_by_class_name('b-search__table')
    i=0
    while input_answer!=myKey and i<5:
        time.sleep(1)
        searchForm.find_element_by_class_name('b-form-button__input').click()
        print("==ИзУрла:"+input_answer)
        print("==НуженКлюч:"+myKey)        
        time.sleep(5)
        input_answer=getInput()
        if i==1:
            enterKey(myKey)
        i+=1
    return input_answer

def nowCaptcha():    
    try:
        testCaptcha=browser.find_element_by_class_name("b-popupa__image-td").find_element_by_class_name("b-popupa__image")
        if testCaptcha.is_displayed() is False:
            #print('testCaptcha_not_displayed')
            return False
        print(testCaptcha.get_attribute("outerHTML"))
        return True
    except NoSuchElementException:
        return False
    
    
def checkCaptcha():
    captchaTry=7
    while ((nowCaptcha() is True) and (captchaTry>0)):
        time.sleep(2)
        captchalink = getCaptchaLink()
        downloadCaptcha(captchalink)
        try:
            captchaWord=getCaptchaWords()
            enterCaptcha(captchaWord)
        except:
            browser.refresh()            
        time.sleep(4)    
        #nowCaptcha()
        print ('captchaTry:'+str(captchaTry))
        captchaTry=captchaTry-1    

def enterKey(myKey):
    time.sleep(3)
    searchForm = browser.find_element_by_class_name('b-search__table')
    enterForm = searchForm.find_element_by_class_name('b-form-input__input')
    enterForm.clear()
    enterForm.send_keys(myKey)
    time.sleep(2)
    searchForm.find_element_by_class_name('b-form-button__input').click()
    time.sleep(2)        
        
    #myKeys=['нск диджитал новосибирск','зарядное устройство для автомобильного аккумулятора в новосибирске']
myKeys=Zapros.objects.all()
for myKey in myKeys:
    myKey=str(myKey)
    if isWordstatStatActual(myKey,actual_date_id) is False:
        countKeys()
        authorization(user1,pass1)     

        checkCaptcha()

        enterKey(myKey)
        
        checkCaptcha()
        
        messageZagruzka = browser.find_element_by_class_name('b-popupa__message')
        if messageZagruzka.is_displayed() is True and messageZagruzka.get_attribute("innerHTML")=='Загрузка...':
            browser.refresh()
            time.sleep(5)
        
        if count.state%10==2:
            time.sleep(25)
            
        if countKeys.state%25==0:
            print("Ключей спарсено="+str(countKeys.state)+" "+str(dt.now()))

        zapros_name = Zapros.objects.get(zapros_name=myKey)
        
        input_answer=checkInput(myKey)
        
           
        try:
            site_answer=browser.find_element_by_class_name('b-history__query').get_attribute("innerHTML")            
        except NoSuchElementException:
            try:
                searchForm.find_element_by_class_name('b-form-button__input').click()
                time.sleep(5)
            except:
                pass
        
        if site_answer.find(str(zapros_name),0,)>-1:
            print('good '+myKey)
            tabl=browser.find_elements_by_class_name("b-history__table-body")

            for x in range(2):
                if x==0:
                    z=-5
                else:
                    z=0
                trs=tabl[x].find_elements_by_tag_name("tr")

                for tr in trs[z:]:
                    date,value = GetStat(tr) #(trs[-1])
                    date1=dt(int(date[-4:]),int(date[3:5]),1)
                    pub_date_id = Pubdate.objects.get(pub_date=date1).id                    
                    wordstatstat, created = WordstatStat.objects.update_or_create(zapros_name=zapros_name, pub_date_id=pub_date_id,
                        defaults=dict(ws_stat_value=value, from_history=1))
                    if pub_date_id>fresh_date_id:
                        fresh_date_id=pub_date_id
        elif site_answer=='Искомая комбинация слов нигде не встречается.' and input_answer==myKey:
            #print('==нет данных '+myKey)
            value=0
            for pub_date_id in range(fresh_date_id-13,fresh_date_id+1):
                wordstatstat, created = WordstatStat.objects.update_or_create(zapros_name=zapros_name, pub_date_id=pub_date_id,
                        defaults=dict(ws_stat_value=value, from_history=1))        
        else:
            print('!!!Лажа '+myKey)
            break
            searchForm.find_element_by_class_name('b-form-button__input').click()
            time.sleep(5)
            browser.refresh()
            time.sleep(5)   
        #print(a+' ' + str(b)) myKey
#browser.close()
print("Done")
print(dt.now())
print("Капчи="+str(count.state))
print("Ключи="+str(countKeys.state))
#25 min 95 keys
#AnticaptchaException: [ERROR_CAPTCHA_UNSOLVABLE:12]Captcha could not be solved by 5 different workers

2018-04-20 13:08:20.005667
Done
2018-04-20 13:10:06.467326
Капчи=0
Ключи=0


In [7]:
site_answer=browser.find_element_by_class_name('b-history__query').get_attribute("innerHTML") 
site_answer

'История показов по фразе «формикарий»'

In [8]:
input_answer=browser.find_element_by_class_name('b-form-input__input').get_attribute("innerHTML")
curl=browser.current_url
word=curl[curl.find('words=',)+6:]
word


#word.encode('utf8')
from urllib.parse import unquote
word = unquote(word)
word

'формикарий'

In [3]:
wrs = Zapros.objects.all()


In [95]:
import pandas as pd
#from django_pandas.io import read_frame
import numpy as np

c1=[]
a=['name']
i=0
for wr in wrs:
    #if i>10:
    #    break
    #print(type(wr))
    #filterargs = {  'zapros_name': wr , 'pub_date_id': 25}
    #sttt=WordstatStat.objects.get**filterargs)

    stat = WordstatStat.objects.filter(zapros_name = wr)
    c=[]
    summar=0
    c.append(str(wr))
    #print (stat(pub_date_id=25).ws_stat_value) 
    
    
    for stt in stat:        
        if stt.pub_date_id>18:
        #print(str(stt.pub_date)[-4:])
            c.append(stt.ws_stat_value)
            summar+=stt.ws_stat_value
            if i==0:
                a.append(str(stt.pub_date))
            #print(stt.pub_date_id)
            if stt.pub_date_id==25 and summar==0:
                stt.delete()
            if stt.pub_date_id==25 and summar>0:
                c.append(summar)                
                c1.append(c)
                #if i<1:
                #    c1.append(c)
                    
            #print(stt.pub_date)
        #a.append(stt.ws_stat_value)
        #b.append(stt.ws_stat_value+2)
    #if summar==0:
        #.delete()

    #c1.append(c)
    i+=1
a.append('summ')    
mys = pd.DataFrame(c1, columns=a)
mys1 = pd.DataFrame(columns=a)
mys=mys.sort_values(by=['summ','01.03.2018','01.02.2018'], ascending=[False,False,False])
#by=['summ','01.03.2018','01.02.2018'], ascending=[False,False,False])
#print(mys)
print(mys.shape[0])
for i in range (1,mys.shape[0]):
    if mys['summ'][i]==mys['summ'][i-1]:
        print(mys['summ'][i])
        mys1=mys1.append((mys[mys.name==mys['name'][i]]))
        
print(mys1)

99
Empty DataFrame
Columns: [name, 01.09.2017, 01.10.2017, 01.11.2017, 01.12.2017, 01.01.2018, 01.02.2018, 01.03.2018, summ]
Index: []


In [90]:
mys1 = pd.DataFrame(columns=a)
for i in range (1,mys.shape[0]):
    if mys['summ'][i]==mys['summ'][i-1]:
        print(mys['summ'][i])
        mys1=mys1.append((mys[mys.name==mys['name'][i]]))
        
print(mys1)

4470
                       name  01.09.2017  01.10.2017  01.11.2017  01.12.2017  \
0  нск диджитал новосибирск         568         562         644        1006   
1  нск диджитал новосибирск         568         562         644        1006   

   01.01.2018  01.02.2018  01.03.2018  summ  
0         611         583         496  4470  
1         611         583         496  4470  


In [91]:
import os
import sys
sys.path.append('/media/sf__Metr-analyzer/metr_django/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'metr.settings'
import django
django.setup()
from yametr.models import *
#from django_pandas.io import read_frame
#qs = Zapros.objects.all()
#df = read_frame(qs)


In [88]:
import pandas as pd
#from django_pandas.io import read_frame
#my_series = pd.Series([5, 6, 7, 8, 9, 10])
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
print(df)
#qs = Zapros.objects.all()
#df = read_frame(qs)

   A  B
0  1  2
1  3  4


In [4]:
import pandas as pd
d = {"price":pd.Series([1, 2, 3], index=['v1', 'v2', 'v3']),"count": pd.Series([10, 12, 7], index=['v1', 'v2', 'v3'])}
df1 = pd.DataFrame(d)
print(df1)

    count  price
v1     10      1
v2     12      2
v3      7      3


In [1]:
import sys     
print(sys.path)

['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-i386-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/home/alex/.local/lib/python3.5/site-packages', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/home/alex/.local/lib/python3.5/site-packages/IPython/extensions', '/home/alex/.ipython']
